#Modelo preditivo de detecção de possíveis inadimplentes


## Conexão com a base no Drive

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import das bibliotecas

In [30]:
#importação de bibliotecas
import pandas as pd 
import numpy as np
import random

## Load do dataset

In [31]:
#importação do csv a ser utilizado
df = pd.read_csv('/content/drive/MyDrive/Hackathon_Inteli_Blockchain/dataset.csv')
df

<ipython-input-31-15dbaaf92efe>:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Hackathon_Inteli_Blockchain/dataset.csv')


,anomes,num_cpf_hash,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,vlr_renda,cod_rating,ind_atrito,ind_engaj,ind_novo_cli
0,202104,fffff8b0db8eff291be8b83f8885f52c52782bb42c3c4b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,202104,ffffd54b45ec46113523184fc07185a0d5cbfa876a07ba...,35943.74,5815.50,NaN,377.0,1.0,NaN,24.0,NaN,10.0,NaN,A,NaN,1.0,NaN
2,202104,ffffd47a92b3e4291c013033ae528708a19eaede50f78e...,6288.22,NaN,NaN,257.0,NaN,NaN,22.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN
3,202104,ffffc102ddd37ec29e985a4564e85a2bace79a85ebff5e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
4,202104,ffffbd4a3d42a12e07b1202d68c33d43220c42c8a55160...,1238.93,1400.00,NaN,773.0,2.0,NaN,21.0,NaN,NaN,NaN,A,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12032488,202204,000055dd71d2394bf27d4fb46f4e8a91ba90a604774953...,970.18,NaN,NaN,415.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12032489,202204,00003a6cf049e4fe3416c1a55d1fc1aac613956ecd5d7a...,0.00,NaN,NaN,689.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12032490,202204,000038baed6e8076adec7272523211faaec7bf8b30b1e1...,30268.92,1631.88,NaN,359.0,1.0,NaN,17.0,NaN,6.0,NaN,A,NaN,1.0,NaN
12032491,202204,00003671431c71ae1a84ec138b63c7a6e465f9616af5ea...,NaN,NaN,NaN,373.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0


## Tratamento do dataset

In [32]:
df.drop(columns=['anomes', 'num_cpf_hash', 'num_produtos', 'num_atend', 'qtd_oper', 'qtd_restr', 'qtd_reclm', 'ind_atrito', 'ind_engaj', 'ind_novo_cli', 'num_atend_atrs', 'vlr_renda', 'vlr_saldo'], inplace=True)

In [33]:
df

,vlr_credito,vlr_score,cod_rating
0,NaN,NaN,NaN
1,35943.74,377.0,A
2,6288.22,257.0,NaN
3,NaN,NaN,NaN
4,1238.93,773.0,A
...,...,...,...
12032488,970.18,415.0,NaN
12032489,0.00,689.0,NaN
12032490,30268.92,359.0,A
12032491,NaN,373.0,NaN


In [34]:
df = df.dropna()
df

,vlr_credito,vlr_score,cod_rating
1,35943.74,377.0,A
4,1238.93,773.0,A
8,81506.54,560.0,A
9,112918.62,374.0,A
16,9806.33,611.0,A
...,...,...,...
12032483,1171.12,575.0,A
12032484,36333.58,538.0,B
12032485,43054.69,437.0,A
12032487,21959.05,410.0,H


risco que o usuário representa - então não é bom dar crédito para ela

In [35]:
df['cod_rating'].value_counts()

A     4406433
H      184518
B       90125
C       89503
D       70967
AA      65273
E       56931
F       49860
HH      49761
G       45158
Name: cod_rating, dtype: int64

Valor do crédito do usuário no mercado

In [36]:
df['vlr_credito'].value_counts()

0.00        96915
0.01          545
200.00        204
400.00        140
300.00        111
            ...  
47595.39        1
24446.94        1
41053.75        1
7897.30         1
21959.05        1
Name: vlr_credito, Length: 3834993, dtype: int64

score do serasa

In [37]:
df['vlr_score'].value_counts()

575.0    204374
562.0     27332
546.0     26905
407.0     26478
397.0     26218
          ...  
869.0         3
849.0         2
64.0          2
60.0          1
63.0          1
Name: vlr_score, Length: 941, dtype: int64

## Mudança da categorização da coluna de rating

In [38]:
mapa_categorias = {
    'AA': 2,
    'A': 2,
    'B': 2,
    'C': 1,
    'D': 1,
    'E': 1,
    'F': 0,
    'G': 0,
    'H': 0,
    'HH': 0
}

df['rating'] = df['cod_rating'].replace(mapa_categorias)
print(df['rating'])

1           2
4           2
8           2
9           2
16          2
           ..
12032483    2
12032484    2
12032485    2
12032487    0
12032490    2
Name: rating, Length: 5108529, dtype: int64


<ipython-input-38-0568ee12cebc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['cod_rating'].replace(mapa_categorias)


Na coluna de rating é apresentado uma escala de 0 a 2, o qual referencia na coluna cod_rating. Sendo dividido da seguinte forma:
- 2 = AA, A, B 
- 1 = C, D, E
- 0 = F, G, H, HH

In [39]:
df

,vlr_credito,vlr_score,cod_rating,rating
1,35943.74,377.0,A,2
4,1238.93,773.0,A,2
8,81506.54,560.0,A,2
9,112918.62,374.0,A,2
16,9806.33,611.0,A,2
...,...,...,...,...
12032483,1171.12,575.0,A,2
12032484,36333.58,538.0,B,2
12032485,43054.69,437.0,A,2
12032487,21959.05,410.0,H,0


In [40]:
df['cod_rating'].value_counts()

A     4406433
H      184518
B       90125
C       89503
D       70967
AA      65273
E       56931
F       49860
HH      49761
G       45158
Name: cod_rating, dtype: int64

dropando a coluna original de rating 

In [41]:
df = df.drop(columns=['cod_rating'])

resetando o index

In [42]:
df = df.reset_index(drop=True)

In [43]:
df

,vlr_credito,vlr_score,rating
0,35943.74,377.0,2
1,1238.93,773.0,2
2,81506.54,560.0,2
3,112918.62,374.0,2
4,9806.33,611.0,2
...,...,...,...
5108524,1171.12,575.0,2
5108525,36333.58,538.0,2
5108526,43054.69,437.0,2
5108527,21959.05,410.0,0


## Fazendo a normatização dos dados, entre 0 e 1

In [44]:
for column in df.columns:
    max_value = df[column].max()
    min_value = df[column].min()
    df[column] = (df[column] - min_value) / (max_value - min_value)
print(df)

         vlr_credito  vlr_score  rating
0           0.003473      0.377     1.0
1           0.000120      0.773     1.0
2           0.007876      0.560     1.0
3           0.010912      0.374     1.0
4           0.000948      0.611     1.0
...              ...        ...     ...
5108524     0.000113      0.575     1.0
5108525     0.003511      0.538     1.0
5108526     0.004161      0.437     1.0
5108527     0.002122      0.410     0.0
5108528     0.002925      0.359     1.0

[5108529 rows x 3 columns]


In [45]:
df['rating'].value_counts()

1.0    4561831
0.0     329297
0.5     217401
Name: rating, dtype: int64

## Exportando para csv o novo dataframe

In [47]:
df

,vlr_credito,vlr_score,rating
0,0.003473,0.377,1.0
1,0.000120,0.773,1.0
2,0.007876,0.560,1.0
3,0.010912,0.374,1.0
4,0.000948,0.611,1.0
...,...,...,...
5108524,0.000113,0.575,1.0
5108525,0.003511,0.538,1.0
5108526,0.004161,0.437,1.0
5108527,0.002122,0.410,0.0


In [46]:
df.to_csv('new_dataframe_c.csv', index=False)